In [ ]:
# Import findspark and initialize.
import findspark
findspark.init()

In [1]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

FileNotFoundError: [WinError 2] The system cannot find the file specified

In [ ]:
# 1. Read the home_sales_revised.csv from the provided AWS S3 bucket location into a PySpark DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"


In [ ]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales")


In [ ]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
query = """
SELECT date_built, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 4
GROUP BY date_built
ORDER BY date_built
"""

four_bedroom_avg_price = spark.sql(query)
four_bedroom_avg_price.show()

In [ ]:
# 4. What is the average price of a home for each year the home was built,
# that has 3 bedrooms and 3 bathrooms, rounded to two decimal places?

query = """
SELECT date_built, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY date_built
ORDER BY date_built
"""

three_bed_avg_price = spark.sql(query)
three_bed_avg_price.show()


In [ ]:
# 5. What is the average price of a home for each year the home was built,
# that has 3 bedrooms, 3 bathrooms, with two floors,
# and is greater than or equal to 2,000 square feet, rounded to two decimal places?

query = """
SELECT date_built, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND square_feet >= 2000
GROUP BY date_built
ORDER BY date_built
"""

large_three_bed_avg_price = spark.sql(query)
large_three_bed_avg_price.show()


In [ ]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

query = """
SELECT view, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view DESC
"""

high_value_homes = spark.sql(query)
high_value_homes.show()

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# 7. Cache the temporary table home_sales.
spark.sql("CACHE TABLE home_sales")

In [ ]:
# 8. Check if the table is cached.
# made it a little easier to see if cached or not
is_cached = spark.catalog.isCached("home_sales")
print(f"Is 'home_sales' table cached? {is_cached}")

In [ ]:
# 9. Using the cached data, run the last query above that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

query = """
SELECT view, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view DESC
"""

cached_high_value_homes = spark.sql(query)
cached_high_value_homes.show()

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# 10. Partition by the "date_built" field on the formatted Parquet home sales data.

df.write.partitionBy("date_built").parquet("home_sales_partitioned", mode="overwrite")

In [ ]:
# 11. Read the formatted Parquet data.
parquet_df = spark.read.parquet("home_sales_partitioned")

# Show the first few rows to confirm successful loading
parquet_df.show(5)

In [ ]:
# 12. Create a temporary table for the Parquet data.
parquet_df.createOrReplaceTempView("home_sales_parquet")

In [ ]:
# 13. Using the Parquet DataFrame, run the last query that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

query = """
SELECT view, ROUND(AVG(price), 2) AS avg_price
FROM home_sales_parquet
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view DESC
"""

parquet_high_value_homes = spark.sql(query)
parquet_high_value_homes.show()

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# 14. Uncache the home_sales temporary table.
spark.sql("UNCACHE TABLE home_sales")

In [ ]:
# 15. Check if the home_sales table is no longer cached.
is_cached = spark.catalog.isCached("home_sales")
print(f"Is 'home_sales' table still cached? {is_cached}")